In [79]:
import os 
import json
import glob
import gzip
import pandas as pd 

In [61]:
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')

In [62]:
# setting template names
gwas_tpl = '' # WashU not available yet
coloc_tpl = 'results/main/GRCh37/coloc/Results/*/*/*/*/FINAL_Summary_Coloc_Gene_SNP_Pairs.longrange.bed.gz'
loop_tpl = 'results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/*/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01_WashU.bed.gz'

## Getting all WashU Files

#### Coloc Files

In [63]:
coloc_data = []
for coloc in glob.glob(coloc_tpl):
    
    run_info = coloc.split('/')
    gwas_source = run_info[6]
    eqtl_source = run_info[7]
    ge_source = run_info[8]   
    if eqtl_source == 'ImmuNexUT':
        eqtl_db = 'ImmuNexUT'
    else:
        eqtl_db = 'eQTL_Catalogue'    
    
    coloc_data.append([gwas_source, eqtl_db, eqtl_source, ge_source, coloc])
coloc_df = pd.DataFrame(coloc_data)
coloc_df.columns = ['gwas_source', 'eqtl_db', 'eqtl_source', 'ge_source', 'file']

In [64]:
coloc_df.shape

(165, 5)

In [65]:
s = coloc_df.file.values.tolist()

In [66]:
print('\n'.join(s))

results/main/GRCh37/coloc/Results/eQTL_Catalogue/T1D_34012112_Gaulton/BLUEPRINT/T-cell/FINAL_Summary_Coloc_Gene_SNP_Pairs.longrange.bed.gz
results/main/GRCh37/coloc/Results/eQTL_Catalogue/T1D_34012112_Gaulton/BLUEPRINT/neutrophil/FINAL_Summary_Coloc_Gene_SNP_Pairs.longrange.bed.gz
results/main/GRCh37/coloc/Results/eQTL_Catalogue/T1D_34012112_Gaulton/Quach_2016/monocyte_R848/FINAL_Summary_Coloc_Gene_SNP_Pairs.longrange.bed.gz
results/main/GRCh37/coloc/Results/eQTL_Catalogue/T1D_34012112_Gaulton/Quach_2016/monocyte_Pam3CSK4/FINAL_Summary_Coloc_Gene_SNP_Pairs.longrange.bed.gz
results/main/GRCh37/coloc/Results/eQTL_Catalogue/T1D_34012112_Gaulton/van_de_Bunt_2015/pancreatic_islet/FINAL_Summary_Coloc_Gene_SNP_Pairs.longrange.bed.gz
results/main/GRCh37/coloc/Results/eQTL_Catalogue/T1D_34012112_Gaulton/Schmiedel_2018/Treg_memory/FINAL_Summary_Coloc_Gene_SNP_Pairs.longrange.bed.gz
results/main/GRCh37/coloc/Results/eQTL_Catalogue/T1D_34012112_Gaulton/Schmiedel_2018/B-cell_naive/FINAL_Summary_Col

#### Loop Files

In [67]:
loop_data = []
for loop in glob.glob(loop_tpl):
    cline = loop.split('/')[5]
    loop_data.append([cline, loop])
loop_df = pd.DataFrame(loop_data)
loop_df.columns = ['cline', 'file']

In [68]:
loop_short_list = ['CD4N', 'CD8N', 'NB', 'CM', 'NK']
loop_df = loop_df.loc[loop_df.cline.isin(loop_short_list)]

In [69]:
loop_df.shape

(5, 2)

In [70]:
loop_df

,cline,file
3,CD4N,results/main/2021_Nikhil_eQTL/Data/FitHiChIP_L...
6,CD8N,results/main/2021_Nikhil_eQTL/Data/FitHiChIP_L...
8,NB,results/main/2021_Nikhil_eQTL/Data/FitHiChIP_L...
10,CM,results/main/2021_Nikhil_eQTL/Data/FitHiChIP_L...
11,NK,results/main/2021_Nikhil_eQTL/Data/FitHiChIP_L...


## Making the Hub

In [71]:
ge_cline_abbrevs = [['Th1-17_memory', 'T-cell'],
                    ['Th2_memory', 'T-cell'],
                    ['Th1_memory', 'T-cell'],
                    ['Treg_naive', 'T-cell'],
                    ['Treg_memory', 'T-cell'],
                    ['T-cell', 'T-cell'],
                    ['Tfh_memory', 'T-cell'],
                    ['Th17_memory', 'T-cell'],
                    ['NK-cell_naive', 'NK-cell'],
                    ['NK', 'NK-cell'],
                    ['USM_B', 'B-cell'],
                    ['Naive_B', 'B-cell'],
                    ['SM_B', 'B-cell'],
                    ['DN_B', 'B-cell'],
                    ['monocyte', 'Monocyte'],
                    ['CL_Mono', 'Monocyte'],
                    ['Int_Mono', 'Monocyte'],
                    ['NC_Mono', 'Monocyte'],
                    ['Naive_CD8', 'CD8 T-cell'],
                    ['Mem_CD8', 'CD8 T-cell'],
                    ['CM_CD8', 'CD8 T-cell'],
                    ['CD8_T-cell_anti-CD3-CD28', 'CD8 T-cell'],
                    ['TEMRA_CD8', 'CD8 T-cell'],
                    ['Th17', 'CD8 T-cell'],
                    ['Tfh', 'CD8 T-cell'],
                    ['Th1', 'CD8 T-cell'],
                    ['Th2', 'CD8 T-cell'],
                    ['EM_CD8', 'CD8 T-cell'],
                    ['Naive_CD4', 'CD4 T-cell'],
                    ['Mem_CD4', 'CD4 T-cell'],
                    ['Fr_I_nTreg', 'CD4 T-cell'],
                    ['Fr_II_eTreg', 'CD4 T-cell'],
                    ['Fr_III_T', 'CD4 T-cell']]

ge_cline_abbrevs = pd.DataFrame(ge_cline_abbrevs)
ge_cline_abbrevs.columns = ['ge_cline', 'abbrev_cline']
#ge_cline_abbrevs.set_index('ge_cline', inplace=True)
ge_cline_abbrevs

,ge_cline,abbrev_cline
0,Th1-17_memory,T-cell
1,Th2_memory,T-cell
2,Th1_memory,T-cell
3,Treg_naive,T-cell
4,Treg_memory,T-cell
5,T-cell,T-cell
6,Tfh_memory,T-cell
7,Th17_memory,T-cell
8,NK-cell_naive,NK-cell
9,NK,NK-cell


In [72]:
# loop_cline_abbrevs = [['CD4_T-cell_naive', 'CD4 T-cell'],
#                          ['NK-cell_naive', 'NK-cell'],
#                          ['CD8_T-cell_naive', 'CD8 T-cell'],
#                          ['monocyte_naive', 'Monocyte'],
#                          ['B-cell_naive', 'B-cell']]
loop_cline_abbrevs = [['CD4N', 'CD4 T-cell'],
                         ['NK', 'NK-cell'],
                         ['CD8N', 'CD8 T-cell'],
                         ['CM', 'Monocyte'],
                         ['NB', 'B-cell']]
loop_cline_abbrevs = pd.DataFrame(loop_cline_abbrevs)
loop_cline_abbrevs.columns = ['loop_cline', 'abbrev_cline']
#loop_cline_abbrevs.set_index('loop_cline', inplace=True)
loop_cline_abbrevs

,loop_cline,abbrev_cline
0,CD4N,CD4 T-cell
1,NK,NK-cell
2,CD8N,CD8 T-cell
3,CM,Monocyte
4,NB,B-cell


In [100]:
def create_washu_longrange_dict(name, file):
    d = {"type": "longrange",
         "showOnHubLoad": True,
         "name": name,
         "filename": file,
         "options": {"displayMode": "arc"}}
    return(d)

def create_washu_bed_dict(name, file):
    d = {"type": "bed",
         "showOnHubLoad": True,
         "name": name,
         "filename": file}
    return(d)

def create_washu_vcf_dict(name, file):
    d = {"type": "vcf",
         "showOnHubLoad": True,
         "name": name,
         "filename": file}
    return(d)

def create_washu_geneAnnotation_dict(name, file):
    d = {"type": "refbed",
         "showOnHubLoad": True,
         "name": name,
         "filename": file}
    return(d)

In [101]:
def zcount(filename):
    with gzip.open(filename) as f:
        return(len(f.readlines()))

In [102]:
for cline in loop_cline_abbrevs.abbrev_cline.unique():
    
    print(cline)
    
    outdir = 'results/main/hubs/loop_plus_coloc_hub/{}'.format(cline.replace(' ', '_'))
    os.makedirs(outdir, exist_ok=True)
    
    json_files = []
    
    # hard link over the gencode tracks
    gencode_path = 'results/refs/washu/gencode.washu.refbed.gz'
    new_basename = os.path.basename(gencode_path)
    old_path = gencode_path
    old_index_path = gencode_path + '.tbi'
    new_path = os.path.join(outdir, new_basename)
    new_index_path = new_path + '.tbi'

    if os.path.exists(old_path) and os.path.exists(old_index_path):
        print('gencode')
        if not os.path.exists(new_path) and not os.path.exists(new_index_path):
            
            print('gencode')
            print(old_path, new_path)
    
            os.link(old_path, new_path)
            os.link(old_index_path, new_index_path)

        # add the file to the json hub config
        d = create_washu_geneAnnotation_dict(new_basename, new_basename)
        json_files.append(d)
    
    
    # get ge-based clines
    ge_clines = ge_cline_abbrevs.loc[ge_cline_abbrevs['abbrev_cline'] == cline]
    loop_clines = loop_cline_abbrevs.loc[loop_cline_abbrevs['abbrev_cline'] == cline]
    
    # get loop files (ready)
    loop_files = loop_df.loc[loop_df['cline'].isin(loop_clines.loop_cline)]
    if len(loop_files) > 0:
        for i, sr in loop_files.iterrows():
            
            new_basename = 'loops.{}.bed.gz'.format(sr.cline)
            new_path = os.path.join(outdir, new_basename)
        
            # hard link over the file
            old_path = sr.file
            old_index_path = sr.file + '.tbi'
            new_path = os.path.join(outdir, new_basename)
            new_index_path = new_path + '.tbi'

            if os.path.exists(old_path) and os.path.exists(old_index_path):
                if not os.path.exists(new_path) and not os.path.exists(new_index_path):
                    os.link(old_path, new_path)
                    os.link(old_index_path, new_index_path)
                    
                # add the file to the json hub config
                d = create_washu_longrange_dict(new_basename, new_basename)
                json_files.append(d)
                
                
    # get coloc files (ready)
    coloc_files = coloc_df.loc[coloc_df['ge_source'].isin(ge_clines.ge_cline)]
    if len(coloc_files) > 0:      
        for i, sr in coloc_files.iterrows():
                        
            new_basename = 'colocs.{}.{}.{}.{}.longrange.bed.gz'.format(sr. gwas_source, sr.eqtl_db,
                                                              sr.eqtl_source, sr.ge_source)
            new_path = os.path.join(outdir, new_basename)
            
            # hard link over the file
            old_path = sr.file
            
            
            num_lines = zcount(sr.file)
            if num_lines == 0:
                continue
            
            old_index_path = sr.file + '.tbi'
            new_path = os.path.join(outdir, new_basename)
            new_index_path = new_path + '.tbi'
            
            if os.path.exists(old_path) and os.path.exists(old_index_path):
                                
                if not os.path.exists(new_path) and not os.path.exists(new_index_path):
                                        
                    os.link(old_path, new_path)
                    os.link(old_index_path, new_index_path)
            
                # add the file to the json hub config
                d = create_washu_longrange_dict(new_basename, new_basename)
                json_files.append(d)
                
    hub_path = os.path.join(outdir, 'hub.config.json')
    with open(hub_path, 'w') as fw:
        print(json.dumps(json_files, indent='\t'), file=fw)
        

CD4 T-cell
gencode
gencode
results/refs/washu/gencode.washu.refbed.gz results/main/hubs/loop_plus_coloc_hub/CD4_T-cell/gencode.washu.refbed.gz
NK-cell
gencode
gencode
results/refs/washu/gencode.washu.refbed.gz results/main/hubs/loop_plus_coloc_hub/NK-cell/gencode.washu.refbed.gz
CD8 T-cell
gencode
gencode
results/refs/washu/gencode.washu.refbed.gz results/main/hubs/loop_plus_coloc_hub/CD8_T-cell/gencode.washu.refbed.gz
Monocyte
gencode
gencode
results/refs/washu/gencode.washu.refbed.gz results/main/hubs/loop_plus_coloc_hub/Monocyte/gencode.washu.refbed.gz
B-cell
gencode
gencode
results/refs/washu/gencode.washu.refbed.gz results/main/hubs/loop_plus_coloc_hub/B-cell/gencode.washu.refbed.gz
